In [10]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score, explained_variance_score

In [2]:
RC_circuit_db = pd.read_csv('./datasets/db_completa_RC_Tecnologia.csv')
RC_circuit_db.head()

,time,voltaje_ent,voltaje_cap_5,R_5,C_5,voltaje_cap_8,R_8,C_8,voltaje_cap_10,R_10,C_10,voltaje_cap_12,R_12,C_12,voltaje_cap_15,R_15,C_15
0,0,4.93,0.00,55300,0.0001,0.00,1750,0.0047,0.0,99300,0.0001,0.0,53100,0.00022,0.00,14920,0.001
1,200,4.93,0.00,55300,0.0001,0.12,1750,0.0047,0.0,99300,0.0001,0.0,53100,0.00022,0.00,14920,0.001
2,400,4.93,0.02,55300,0.0001,0.30,1750,0.0047,0.0,99300,0.0001,0.0,53100,0.00022,0.00,14920,0.001
3,600,4.93,0.24,55300,0.0001,0.50,1750,0.0047,0.0,99300,0.0001,0.0,53100,0.00022,0.00,14920,0.001
4,800,4.93,0.43,55300,0.0001,0.66,1750,0.0047,0.0,99300,0.0001,0.0,53100,0.00022,0.04,14920,0.001


In [3]:
# Funcion de normalizacion
def normalize(x, xmin, xmax):
    return (xmax - xmin)*((x - np.min(x))/(np.max(x) - np.min(x))) + xmin

In [4]:
#X = [tiempo, R, C, Ve]
#y = [Vc]


R = normalize(np.hstack([RC_circuit_db['R_5'], RC_circuit_db['R_8'], RC_circuit_db['R_10'], RC_circuit_db['R_12'], RC_circuit_db['R_15']]), 0.1, 0.8)
C = normalize(np.hstack([RC_circuit_db['C_5'], RC_circuit_db['C_8'], RC_circuit_db['C_10'], RC_circuit_db['C_12'], RC_circuit_db['C_15']]), 0.1, 0.8)
t = normalize(np.hstack([RC_circuit_db['time'], RC_circuit_db['time'], RC_circuit_db['time'], RC_circuit_db['time'], RC_circuit_db['time']]), 0.1, 0.8)
Ve = np.hstack([RC_circuit_db['voltaje_ent'], RC_circuit_db['voltaje_ent'], RC_circuit_db['voltaje_ent'], RC_circuit_db['voltaje_ent'], RC_circuit_db['voltaje_ent']])

# Input variables
X = np.vstack([R, C, t, Ve]).T

# Target
y = np.hstack([RC_circuit_db['voltaje_cap_5'], RC_circuit_db['voltaje_cap_8'], RC_circuit_db['voltaje_cap_10'], RC_circuit_db['voltaje_cap_12'], RC_circuit_db['voltaje_cap_15']])

In [5]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.30, random_state = 2002)

print(X_train.shape)
print(X_test.shape)
print(y_train.shape)
print(y_test.shape)

(14000, 4)
(6000, 4)
(14000,)
(6000,)


In [6]:
class ANN():
    def __init__(self, X, Y, n_neuronas):
        self.x = X
        n_entradas = X.shape[1]
        self.IW = np.random.uniform(size = [n_entradas, n_neuronas])
        self.b = np.random.uniform(size = [n_neuronas])
        self.y = Y

    # Funciones de activacion
    def logsig(self, n):
        return (1/(1 + np.exp(-n)))

    def tansig(self, n):
        return (2/(1 + np.exp(-2*n))) - 1

    def relu(self, n):
        return np.maximum(0, n)

    def train(self):
        self.LW = np.dot(np.linalg.pinv(self.logsig(self.x.dot(self.IW) + self.b)), self.y)

    def predict(self, Xdata):
        self.x = Xdata
        return np.dot(self.logsig(self.x.dot(self.IW) + self.b), self.LW)

In [8]:
def regression_results(y_true, y_pred):
    # Regression metrics
    ev = explained_variance_score(y_true, y_pred)
    mae = mean_absolute_error(y_true, y_pred) 
    mse = mean_squared_error(y_true, y_pred) 
    r2 = r2_score(y_true, y_pred)

    print('explained_variance: ', round(ev,4))    
    print('MAE: ', round(mae,4))
    print('MSE: ', round(mse,4))
    print('R²: ', round(r2,4))

In [ ]:
# X [k_muestras x n_variables_entrada]
# y [k_muestras]

r2 = 0
goal = 0.99
n_neuronas = 1

while r2 < goal:
    model = ANN(X_train, y_train, n_neuronas)
    model.train()
    y_train_pred = model.predict(X_train)

    r2 = r2_score(y_train, y_train_pred)

    n_neuronas += 1

print("Best model with %d neurons" % (n_neuronas))

y_train_pred = model.predict(X_train)
y_test_pred = model.predict(X_test)

regression_results(y_train, y_train_pred)
regression_results(y_test, y_test_pred)

#print("RMSE (train):", np.sqrt(np.mean( (y_train-y_train_pred)**2) ))
#print("RMSE (test):", np.sqrt(np.mean( (y_test-y_test_pred)**2) ))

In [22]:
n_neuronas

30